# Market marker - price and fill orders

Inspo:
- https://github.com/cornwarecjp/marketmaker/blob/master/marketmaker.py
- https://money.stackexchange.com/questions/24442/how-are-futures-contracts-setup-at-an-exchange
- https://pdfs.semanticscholar.org/e0f7/35e27f5f89ecc6dd7d7cad8f6410aff6270f.pdf
- https://www.researchgate.net/profile/Bastien-Baldacci/publication/334760288_Algorithmic_market_making_the_case_of_equity_derivatives/links/5e1cf0974585159aa4ce78b8/Algorithmic-market-making-the-case-of-equity-derivatives.pdf

3 possibilities for liquidity:
1. Treat the underlying asset (the music time series) as the actual asset value $S_t$, which is not tradeable - like weather data. Issue contracts on this underlying non-tradeable asset and build the market around that.
2. Issue shares in predictions for the underlying asset according to a stochastic process, auction off to agents, and also play the role of market-maker
    - At some point, could jointly optimize the stochastic process, the auction structure and market-making mechanism (simplest: Hanson's LMSR).
    - Prep for blockchain style model.
3. Fully predictive approach: LMSR where you buy shares for predicting outcome A vs. outcome B. At the end of the outcome, financial reward/loss.
    - Number of outstanding shares (corresponding to discrete predicted outcomes) starts at 0 and only grows.
        - https://mason.gmu.edu/~rhanson/mktscore.pdf
        - http://blog.oddhead.com/2006/10/30/implementing-hansons-market-maker/ - for selling too
    - Could build derivatives on top of this later, e.g. make shares tradeable + right to buy/sell share later for given price.
    
Focus on #3 for now - the most feasible.
1. Start with simplest decision problem: predict UP vs. DOWN (buying N shares of up vs. down) for K periods from now.

In [1]:
from __future__ import division
from more_itertools import peekable
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy as scp
import pickle
import magenta
import os, time, re, json
%matplotlib inline
from IPython.core.display import display, HTML
### change width of notebook display
display(HTML("<style>.container { width:70% !important; }</style>"))

import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

# for exposing API
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
from flask import Flask
from flask_restful import Resource, Api

JUPYTER_PICKLE_FILE = "config/shared_jupyter_data.pkl"
def write_shared_jupyter(key, value, path=JUPYTER_PICKLE_FILE, overwrite=False):
    if (os.path.exists(path)):
        with open(path, "rb") as fp:
            shared_jupyter_data = pickle.load(fp)
        if overwrite:
            shared_jupyter_data = {key: value}
        else:
            shared_jupyter_data[key] = value
    else:
        shared_jupyter_data = {key: value}
    with open(path, 'wb') as fp: 
        pickle.dump(shared_jupyter_data, fp)

def read_shared_jupyter(key=None, path=JUPYTER_PICKLE_FILE):
    if (os.path.exists(path)):
        with open(path, "rb") as fp:
            shared_jupyter_data = pickle.load(fp)
            if key is not None:
                if key in shared_jupyter_data:
                    return(shared_jupyter_data[key])
                else:
                    print("Not found!")
                    return(None)
            else:
                return(shared_jupyter_data)
    else:
        print("No data")

def pandasToJson(df):
    return(df.to_json(orient="split"))
def jsonToPandas(json):
    return(pd.read_json(json, orient="split"))

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

FIG_WIDTH = 1200
FIG_HEIGHT = 800

PITCH_MIN = 20
PITCH_MAX = 120
VELOCITY_MIN = 0
VELOCITY_MAX = 120

def hheader(x):
    print("#########################################")
    print("### {}".format(x))
    print("#########################################")

# Magenta dependencies:
# https://github.com/magenta/magenta

# Magenta uses pretty_midi to deal with midi files
import pretty_midi

C:\Users\echow\Miniconda3\envs\tensorflow\lib\site-packages\librosa\util\decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
C:\Users\echow\Miniconda3\envs\tensorflow\lib\site-packages\librosa\util\decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
C:\Users\echow\Miniconda3\envs\tensorflow\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Could

# Simulate orders for market-making

Orders will be to buy or sell shares of A, B, C ... which are the predicted discrete outcomes.

Start simple by betting whether will go up or down for (horizon) periods from now.

Specifically, a bet is a tuple (quantity, horizon, outcome).

https://lobsterdata.com/info/DataStructure.php

In [2]:
### order book template data
order_book_msgs = pd.read_csv("order_book/AMZN_2012-06-21_34200000_57600000_message_1.csv", header=None)
order_book_msgs.columns = ['ts_secs_after_midnight', "event_type", "order_id", "bet_quantity", "price", "direction"]
order_book_msgs['price'] = order_book_msgs['price'] / 1000.0 # in dollars

### only look at submitted orders for now
order_book_msgs = order_book_msgs.loc[order_book_msgs['event_type'] == 1,].reset_index(drop=True)
order_book_msgs['bet_outcome'] = order_book_msgs['direction'] # +1 = up, -1 = down

### generate random order times (monotonically increasing) - something plausible
# order_book_msgs = order_book_msgs.reset_index()
# order_book_msgs['order_book_streaming_ts'] = order_book_msgs['index']
order_book_msgs['order_book_streaming_ts'] = np.sort(np.random.gamma(1, np.sqrt(order_book_msgs.shape[0]), order_book_msgs.shape[0]))
# start at time 1000 so can weave together streaming order book data and also streaming music data
# this will be set with time.time() in the real stream.
order_book_msgs['order_book_streaming_ts'] = order_book_msgs['order_book_streaming_ts'] + 1000

### simulate bet horizon
order_book_msgs['bet_horizon'] = order_book_msgs['order_book_streaming_ts'] + np.random.randint(20, 30, order_book_msgs.shape[0])

order_book_msgs = order_book_msgs.drop(columns=['ts_secs_after_midnight', 'direction', "price"])

# price here is set by the market-maker entirely (agents are price-takers)
# maybe later can incorporate price-power on the part of the agents

### simulate some agent IDs
### (assume agents have enough wealth)
SIM_N_AGENTS = 100
order_book_msgs['agent_id'] = np.random.randint(0, SIM_N_AGENTS, order_book_msgs.shape[0])
order_book_msgs = order_book_msgs[['order_book_streaming_ts', 'agent_id', 'order_id', 'bet_outcome', 'bet_horizon', 'bet_quantity']]

order_book_msgs.to_csv("order_book/simulated_order_book.csv")

order_book_evolv = pd.read_csv("order_book/AMZN_2012-06-21_34200000_57600000_message_1.csv")

In [3]:
order_book_msgs.head()

,order_book_streaming_ts,agent_id,order_id,bet_outcome,bet_horizon,bet_quantity
0,1000.000774,70,11885113,1,1022.000774,21
1,1000.002259,77,16207239,-1,1023.002259,100
2,1000.003657,65,16208720,-1,1025.003657,50
3,1000.014393,11,16240373,-1,1022.014393,100
4,1000.016015,96,16240856,-1,1022.016015,100


# Automated market-making

- Stream order book and music (underlying) simultaneously
- Price orders
- Fill orders (and reexpose endpoint)

---

Remember, in this simple model, we just issue new shares.

For the real line:
http://yiling.seas.harvard.edu/wp-content/uploads/gao-wine09-interval.pdf

Could use Monte Carlo to approximate those integrals.

In [4]:
""" Set up order book stream """

### for reading in chunks
from collections import deque

def csvStream(csvfile):
    csv_stream = pd.read_csv(csvfile, index_col=0, iterator=True)
    return(csv_stream)

### Always fill 1 order at a time
def nextChunk(csvStream, chunksize=1):
    return(csvStream.get_chunk(chunksize))

### Create order book stream
orderBookStream = csvStream("order_book/simulated_order_book.csv")

In [5]:
""" Automated market-making: set up parameters for market-maker
"""

MARKET_MAKER_WEALTH = 10**6

In [7]:
""" Automated market-making: LMSR
"""

import requests

### LMSR parameters
b = 1000
quantity_shares_up = 0
quantity_shares_down = 0
def execute_order(quantity, bet_outcome="up"):
    if bet_outcome == "up":
        cost_before = 0

def quote_curr_price():
    return()

### Iteration parameters
STREAM_SLEEP_SEC = 1


print("Upper bound on market-maker loss: {}".format(b * np.log(2)))


### Important note: orders need to be able to pile up (so that no order gets skipped)
### while the music stream (itself sampled) can be sampled for Bayesian learning.
### This is because while the market maker observes both the order book stream and
### the music stream, the market maker only needs to directly interact with the order
### book stream, while the latter is primarily used just for learning.

iterations = 1
while True:
    if (iterations % 1000 == 0):
        print("Iterations {} ...".format(iterations+1))
    
    ### Read in music stream (assume perpetual)
    currAssetData = requests.get("http://localhost:8100/read")
    if (currAssetData.status_code != 200):
        raise Exception("GET request failed.")
    currAsset = currAssetData.json()
    currAssetTime = currAsset['currTime']
    currAssetBar = jsonToPandas(currAsset['lastBar'])
    
    ### Read in current order (assume perpetual)
    currOrder = nextChunk(orderBookStream)
    
    print("--------------------------------------------")
    print(currAssetTime)
    display(currAssetBar)
    display(currOrder)
        
    continue

    """ Analysis with current order here """

    ### Calculate price
    
    ### Write out price predictio to shared jupyter space
        
    ### store via Jupyter so other notebooks can access
    ### (later work might actually build an API)
#     write_shared_jupyter("currOrder", currOrder, overwrite=False)
    
    ### read trading decisions from other agents

    ### Take a short break between analyses (so plotly can catch up)
    ### should be >> plot auto-update interval so that all plots
    ### update basically at the same time. 
    time.sleep(STREAM_SLEEP_SEC) # 1 second is comfortable for nice UI
    iterations += 1

Upper bound on market-maker loss: 693.1471805599452
--------------------------------------------
1618537091.9771633


,streaming_start_sec,piece_num,start_sec,start_min,start_mean,start_median,start_max,start_str_concat,end_min,end_mean,end_median,end_max,end_str_concat,pitch_min,pitch_mean,pitch_median,pitch_max,pitch_str_concat,velocity_min,velocity_mean,velocity_median,velocity_max,velocity_str_concat,duration_min,duration_mean,duration_median,duration_max,duration_str_concat,canonical_composer_mode,split_mode,year_mode,total_duration_mode,curr_filename_mode,piece_changed
905,905,0,905,905.00104,905.466111,905.39792,905.96250,"905.00104,905.01146,905.125,905.13542,905.2583...",905.09479,905.589166,905.46667,906.24062,"905.13229,905.09479,905.20938,905.24688,905.35...",38,56.466667,62,74,"42,62,45,66,69,48,38,66,69,47,74,43,64,47,67",69,76.066667,73,88,"70,72,73,69,69,73,82,83,77,88,87,72,76,73,77",0.03438,0.123055,0.084380,0.57604,"0.13125,0.08333,0.08438,0.11146,0.1,0.05833,0....",Johann Sebastian Bach,train,2004,967.16405,2004__MIDI-Unprocessed_SMF_02_R1_2004_01-05_OR...,0
906,906,0,906,906.07396,906.505530,906.50104,906.90938,"906.07396,906.09688,906.20104,906.23125,906.36...",906.13854,906.619231,906.68333,907.03854,"906.13854,906.17396,906.24167,906.37708,906.49...",40,59.615385,66,76,"50,71,40,67,71,48,76,45,66,48,69,52,72",68,80.230769,82,89,"76,82,87,82,80,88,89,79,82,68,70,78,82",0.04063,0.113702,0.077080,0.53750,"0.06458,0.07708,0.04063,0.14583,0.13646,0.0718...",Johann Sebastian Bach,train,2004,967.16405,2004__MIDI-Unprocessed_SMF_02_R1_2004_01-05_OR...,0
907,907,0,907,907.03229,907.455821,907.45104,907.87604,"907.03229,907.04896,907.175,907.175,907.29688,...",907.09479,907.578798,907.57917,907.97708,"907.09479,907.45208,907.21979,907.31042,907.33...",42,61.941176,67,79,"42,69,50,72,48,78,75,47,79,69,50,74,48,71,55,5...",9,72.000000,79,88,"86,84,79,81,79,83,9,83,86,51,60,71,59,78,70,77,88",0.02292,0.122978,0.108330,0.40313,"0.0625,0.40313,0.04479,0.13542,0.03542,0.12708...",Johann Sebastian Bach,train,2004,967.16405,2004__MIDI-Unprocessed_SMF_02_R1_2004_01-05_OR...,0
908,908,0,908,908.01042,908.444297,908.43333,908.97500,"908.01042,908.02708,908.03333,908.13958,908.14...",908.08438,908.565187,908.55417,909.10938,"908.12396,908.14792,908.08438,908.20312,908.24...",50,63.263158,60,77,"77,55,53,59,74,57,60,76,57,72,71,69,54,50,66,6...",61,74.000000,76,89,"88,63,66,68,89,61,79,86,62,61,70,76,64,76,82,7...",0.02917,0.120888,0.113540,0.28229,"0.11354,0.12083,0.05104,0.06354,0.09375,0.0291...",Johann Sebastian Bach,train,2004,967.16405,2004__MIDI-Unprocessed_SMF_02_R1_2004_01-05_OR...,0
909,909,0,909,909.08438,909.510548,909.49844,909.96146,"909.08438,909.10104,909.18958,909.2375,909.362...",909.14271,909.694401,909.61667,910.58750,"909.14271,909.18646,909.37604,909.29479,909.5,...",38,59.437500,63,71,"55,71,55,69,48,64,67,50,67,62,69,38,60,66,43,67",70,81.687500,82,87,"85,82,70,81,77,87,84,82,83,87,80,78,82,83,81,85",0.04375,0.183855,0.120315,0.62604,"0.05833,0.08542,0.18646,0.05729,0.1375,0.06146...",Johann Sebastian Bach,train,2004,967.16405,2004__MIDI-Unprocessed_SMF_02_R1_2004_01-05_OR...,0


,order_book_streaming_ts,agent_id,order_id,bet_outcome,bet_horizon,bet_quantity
1,1000.002259,77,16207239,-1,1023.002259,100


--------------------------------------------
1618537094.581474


,streaming_start_sec,piece_num,start_sec,start_min,start_mean,start_median,start_max,start_str_concat,end_min,end_mean,end_median,end_max,end_str_concat,pitch_min,pitch_mean,pitch_median,pitch_max,pitch_str_concat,velocity_min,velocity_mean,velocity_median,velocity_max,velocity_str_concat,duration_min,duration_mean,duration_median,duration_max,duration_str_concat,canonical_composer_mode,split_mode,year_mode,total_duration_mode,curr_filename_mode,piece_changed
910,910,0,910,910.09896,910.336112,910.246875,910.86354,"910.09896,910.09896,910.24583,910.24792,910.46...",910.15312,910.382985,910.285415,910.92500,"910.15312,910.15729,910.275,910.29583,910.4916...",47,55.000000,56.0,62,"62,47,59,50,55,57",72,79.333333,80.0,84,"84,72,83,77,84,76",0.02917,0.046877,0.051045,0.06146,"0.05417,0.05833,0.02917,0.04792,0.03021,0.06146",Johann Sebastian Bach,train,2004,967.16405,2004__MIDI-Unprocessed_SMF_02_R1_2004_01-05_OR...,0
911,911,0,911,911.01354,911.495704,911.491145,911.97500,"911.01354,911.14375,911.29271,911.42917,911.55...",911.13646,911.611458,911.572915,912.13854,"911.13646,911.28229,911.39479,911.50729,911.63...",50,53.000000,54.0,57,"54,50,55,54,50,57,54,50",65,70.375000,70.0,77,"77,69,65,68,69,72,72,71",0.05208,0.115754,0.112500,0.18333,"0.12292,0.13854,0.10208,0.07812,0.08542,0.1833...",Johann Sebastian Bach,train,2004,967.16405,2004__MIDI-Unprocessed_SMF_02_R1_2004_01-05_OR...,0
912,912,0,912,912.14375,912.513151,912.492185,912.97292,"912.14375,912.16146,912.29688,912.41979,912.56...",912.16250,912.600520,912.600000,913.05312,"912.27917,912.1625,912.3625,912.52812,912.6718...",50,54.625000,54.5,60,"55,50,54,50,57,54,57,60",25,65.375000,70.5,77,"63,25,69,69,75,72,77,73",0.00104,0.087370,0.093750,0.14688,"0.13542,0.00104,0.06562,0.10833,0.10729,0.1468...",Johann Sebastian Bach,train,2004,967.16405,2004__MIDI-Unprocessed_SMF_02_R1_2004_01-05_OR...,0
913,913,0,913,913.11667,913.597742,913.614580,913.99375,"913.11667,913.41354,913.55104,913.67812,913.83...",913.15104,913.654340,913.697400,914.02812,"913.15104,913.47708,913.67188,913.72292,913.87...",50,56.166667,57.0,62,"50,59,55,59,62,52",68,75.666667,76.0,83,"83,80,76,71,76,68",0.03437,0.056595,0.043230,0.12083,"0.03437,0.06354,0.12083,0.04479,0.04167,0.03437",Johann Sebastian Bach,train,2004,967.16405,2004__MIDI-Unprocessed_SMF_02_R1_2004_01-05_OR...,0
914,914,0,914,914.27292,914.618402,914.613540,914.96562,"914.27292,914.41354,914.53958,914.6875,914.831...",914.33646,914.697223,914.686980,915.03542,"914.33646,914.53854,914.61042,914.76354,914.89...",54,59.500000,60.0,64,"60,57,60,64,54,62",71,74.666667,74.0,79,"79,73,75,71,79,71",0.06354,0.078818,0.070310,0.12500,"0.06354,0.125,0.07083,0.07604,0.06771,0.06979",Johann Sebastian Bach,train,2004,967.16405,2004__MIDI-Unprocessed_SMF_02_R1_2004_01-05_OR...,0


...

,order_book_streaming_ts,agent_id,order_id,bet_outcome,bet_horizon,bet_quantity
12,1000.053832,90,16610191,-1,1020.053832,100


--------------------------------------------
1618537115.7985723


,streaming_start_sec,piece_num,start_sec,start_min,start_mean,start_median,start_max,start_str_concat,end_min,end_mean,end_median,end_max,end_str_concat,pitch_min,pitch_mean,pitch_median,pitch_max,pitch_str_concat,velocity_min,velocity_mean,velocity_median,velocity_max,velocity_str_concat,duration_min,duration_mean,duration_median,duration_max,duration_str_concat,canonical_composer_mode,split_mode,year_mode,total_duration_mode,curr_filename_mode,piece_changed
950,950,1,6,6.26250,6.272135,6.273435,6.27917,"6.2625,6.27083,6.27604,6.27917",6.37188,6.464845,6.478125,6.53125,"6.45625,6.53125,6.5,6.37188",60.0,69.50,69.5,79.0,"79,60,67,72",40.0,41.25,40.5,44.0,"44,41,40,40",0.09271,0.19271,0.208855,0.26042,"0.19375,0.26042,0.22396,0.09271",Sergei Rachmaninoff,validation,2004.0,266.40602,2004__MIDI-Unprocessed_SMF_02_R1_2004_01-05_OR...,0
951,951,1,7,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,None,None,None,NaN,NaN,None,0
952,952,1,8,8.49792,8.694168,8.681250,8.87813,"8.49792,8.53438,8.60312,8.62396,8.6625,8.7,8.7...",8.57292,8.779167,8.801560,8.98854,"8.57292,8.58333,8.65938,8.68021,8.77708,8.8260...",27.0,36.60,36.5,46.0,"27,39,29,41,31,43,32,44,34,46",44.0,60.90,62.0,73.0,"59,44,48,60,64,68,58,67,68,73",0.04896,0.08500,0.079165,0.12604,"0.075,0.04896,0.05625,0.05625,0.11458,0.12604,...",Sergei Rachmaninoff,validation,2004.0,266.40602,2004__MIDI-Unprocessed_SMF_02_R1_2004_01-05_OR...,0
953,953,1,9,9.00937,9.019790,9.019790,9.03021,"9.00937,9.03021",9.64479,9.657810,9.657810,9.67083,"9.64479,9.67083",36.0,42.00,42.0,48.0,"36,48",54.0,63.00,63.0,72.0,"54,72",0.63542,0.63802,0.638020,0.64062,"0.63542,0.64062",Sergei Rachmaninoff,validation,2004.0,266.40602,2004__MIDI-Unprocessed_SMF_02_R1_2004_01-05_OR...,0
954,954,1,10,10.55729,10.571090,10.574475,10.57812,"10.55729,10.57187,10.57708,10.57812",12.45104,12.559375,12.547920,12.69062,"12.58438,12.69062,12.51146,12.45104",48.0,54.75,54.5,62.0,"62,48,53,56",56.0,65.75,63.0,81.0,"81,56,60,66",1.87292,1.98828,1.980725,2.11875,"2.02708,2.11875,1.93437,1.87292",Sergei Rachmaninoff,validation,2004.0,266.40602,2004__MIDI-Unprocessed_SMF_02_R1_2004_01-05_OR...,0


,order_book_streaming_ts,agent_id,order_id,bet_outcome,bet_horizon,bet_quantity
13,1000.065071,55,16657800,-1,1024.065071,100


KeyboardInterrupt: 